<font color="#CA3532"><h1 align="left">Master Data Analytics. EDEM.</h1></font>
<font color="#6E6E6E"><h2 align="left">Herramientas MLOps.</h2></font>
<font color="#6E6E6E"><h2 align="left">Tarea 1. Pipeline entrenamiento de modelos.</h2></font>
#### Daniel Ruiz Riquelme
https://docs.metaflow.org/metaflow/basics

##  Install dependencies

In [1]:
!pip install metaflow
!pip install scikit-learn
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.6 MB/s eta 0:00:00


## Set username

In [2]:
# Set username for workflows
import os
os.environ["USERNAME"] = "Mar"

In [9]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')

    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)

    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris

        # Load the iris dataset
        iris = load_iris()

        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member

        self.next(self.split_data)

    @step
    def split_data(self):
        #Split the data into train and test
        # TODO: WRITE YOUR CODE HERE
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=self.random_state
        )

        self.next(self.train)

    @step
    def train(self):
        # Train the model
        # TODO: WRITE YOUR CODE HERE
        self.model = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train)

        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        # Print some metrics
        # TODO: WRITE YOUR CODE HERE
        self.y_pred = self.model.predict(self.X_test)

        self.accuracy = accuracy_score(self.y_test, self.y_pred)
        self.precision = precision_score(self.y_test, self.y_pred, average='weighted')
        self.recall = recall_score(self.y_test, self.y_pred, average='weighted')
        self.f1 = f1_score(self.y_test, self.y_pred, average='weighted')

        print(f"Accuracy: {self.accuracy}")
        print(f"Precision: {self.precision}")
        print(f"Recall: {self.recall}")
        print(f"F1-score: {self.f1}")

        self.next(self.register_model)

    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        # TODO: WRITE YOUR CODE HERE
        joblib.dump(self.model, 'random_forest_model.pkl')

        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    TrainingFlow()

Overwriting metaflow_trainingflow.py


In [10]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.12.5 executing TrainingFlow for user:Mar
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2024-06-22 10:49:51.326 Workflow starting (run-id 1719053391324824):
2024-06-22 10:49:51.333 [1719053391324824/start/1 (pid 4313)] Task is starting.
2024-06-22 10:49:52.515 [1719053391324824/start/1 (pid 4313)] Task finished successfully.
2024-06-22 10:49:52.519 [1719053391324824/ingest_data/2 (pid 4323)] Task is starting.
2024-06-22 10:49:53.634 [1719053391324824/ingest_data/2 (pid 4323)] Task finished successfully.
2024-06-22 10:49:53.638 [1719053391324824/split_data/3 (pid 4333)] Task is starting.
2024-06-22 10:49:54.754 [1719053391324824/split_data/3 (pid 4333)] Task finished successfully.
2024-06-22 10:49:54.758 [1719053391324824/train/4 (pid 4343)] Task is starting.
2024-06-22 10:49:56.061 [1719053391324824/train/4 (pid 4343)] Task finished successfully.
2024-06-22 10:49:56.065 [1719053391324824/show_metrics/5 